In [ ]:
%load_ext aiida
%aiida

In [ ]:
import ipywidgets as ipw
from aiida.orm import StructureData
from datetime import datetime

In [ ]:
qb = QueryBuilder()
qb.append(StructureData, filters={
    "extras": {"has_key": "surfaces"}}
)

In [ ]:
#        filters={
#            "attributes.process_label": {"in": workchain_list},
#            "or": [
#                {"extras": {"!has_key": "preprocess_version"}},
#                {"extras.preprocess_version": {"<": PREPROCESS_VERSION}},
#            ],
#        },
#    )
qb.order_by({StructureData: {"ctime": "asc"}})

In [ ]:
table = ipw.HTML()
display(table)

In [ ]:
# html table header
html = "<style>#aiida_results td,th {padding: 2px}</style>"
html += '<table border=1 id="aiida_results" style="margin:0px"><tr>'
#html += "<th>PK</th>"
html += "<th>Creation Time</th>"
html+= "Calc. Type"
html += "<th >Description</th>"
html += "<th >Thumbnail</th>"
html += "</tr>"

In [ ]:
def thunmnail_raw(thumbnail=None,uuid=None,pk=None,description=''):
    # image with a link to structure export
    
    html = (
        '<td><a target="_blank" href="./export_structure.ipynb?uuid=%s">'
        % uuid
    )
    html += (
        '<img width="100px" src="data:image/png;base64,%s" title="PK%d: %s">'
        % (thumbnail, pk, description)
    )
    html += "</a></td>"
    return html
def link_to_viewer(description='',uuid=''):
    return '<a target="_blank" href="./export_structure.ipynb?uuid=%s"> %s </a></td>'% (uuid,description)
    

In [ ]:
# date type description thumb
for node_tuple in qb.iterall():
    node = node_tuple[0]
    html+= '<tr>' # beginning raw
    #
    html+='<th> %s </th>' % node.ctime.ctime()
    html+='<th>' # start list of calculations done on same geo
    for key in node.extras['surfaces']:
        html+='%s<br>' % key # e.g. "geo_opt"
        html+='</th>'
        html+='<th><ul>'
        for uuid in node.extras['surfaces'][key]: # e.g. list of all geo_opt
            html+='<li>' + link_to_viewer(load_node(uuid).description,uuid) + '</li>'
            
    html+='</ul></th>' # end list of calculations for a specific geo
    html+=thunmnail_raw(thumbnail=node.extras['thumbnail'],uuid=node.uuid,pk=node.pk,description='')
    html+='</th>'
    #
    html+='</tr>' # end raw